In [1]:
import numpy as np
from scipy import ndimage
import cv2 as cv

In [2]:
def canny_edge(image):
    threshold_weak = None
    threshold_strong = None
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    sigma = 5
    kernel_size = 5
    size = int(kernel_size) // 2
    x, y = np.mgrid[-size:size + 1, -size:size + 1]
    normal = 1 / (2.0 * np.pi * sigma ** 2)
    gaussian_kernel = np.exp(-((x ** 2 + y ** 2) / (2.0 * sigma ** 2))) * normal

    image = cv.filter2D(src=image, kernel=gaussian_kernel, ddepth=19)

    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

    Image_x = ndimage.filters.convolve(image, Kx)
    Image_y = ndimage.filters.convolve(image, Ky)

    G = np.hypot(Image_x, Image_y)
    magnitude = G / G.max() * 255
    theta = np.arctan2(Image_y, Image_x)
    max_magnitude = np.max(magnitude)
    if not threshold_weak:
        threshold_weak = max_magnitude * 0.1

    if not threshold_strong:
        threshold_strong = max_magnitude * 0.5

    height, width = image.shape

   
    for x in range(width):
        for y in range(height):

            grad = theta[y, x]
            grad = abs(grad - 180) if abs(grad) > 180 else abs(grad)

        
            if grad <= 22.5:
                n1x, n1y = x - 1, y
                n2x, n2y = x + 1, y

            elif 22.5 < grad <= (22.5 + 45):
                n1x, n1y = x - 1, y - 1
                n2x, n2y = x + 1, y + 1

           
            elif (22.5 + 45) < grad <= (22.5 + 90):
                n1x, n1y = x, y - 1
                n2x, n2y = x, y + 1

      
            elif (22.5 + 90) < grad <= (22.5 + 135):
                n1x, n1y = x - 1, y + 1
                n2x, n2y = x + 1, y - 1

            
            elif (22.5 + 135) < grad <= (22.5 + 180):
                n1x, n1y = x - 1, y
                n2x, n2y = x + 1, y

            
            if width > n1x >= 0 and height > n1y >= 0:
                if magnitude[y, x] < magnitude[n1y, n1x]:
                    magnitude[y, x] = 0
                    continue

            if width > n2x >= 0 and height > n2y >= 0:
                if magnitude[y, x] < magnitude[n2y, n2x]:
                    magnitude[y, x] = 0

    ids = np.zeros_like(image)
    
    for x in range(width):
        for y in range(height):

            grad_mag = magnitude[y, x]

            if grad_mag < threshold_weak:
                magnitude[y, x] = 0
            elif threshold_strong > grad_mag >= threshold_weak:
                ids[y, x] = 1
            else:
                ids[y, x] = 2

    return magnitude

In [3]:
def harris_corners(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dist = cv.cornerHarris(gray, 2, 3, 0.04)
    # result is dilated for marking the corners, not important
    dist = cv.dilate(dist, None)
    # Threshold for an optimal value, it may vary depending on the image.
    image[dist > 0.01 * dist.max()] = [0, 0, 255]
    return image

In [4]:
def visualize_image(frame_name, frame):
    cv.imshow(frame_name, frame)
    cv.imwrite(frame_name + ".jpg", frame)
    if cv.waitKey(0) == ord('q'):
        cv.destroyAllWindows()

In [5]:
if __name__ == "__main__":
    cap = cv.VideoCapture("files/10sec_video.mp4")
    frames_dict = {}
    count = 0
    while cap.read()[0]:
        frames_dict[count] = cap.read()[1]
        count += 1
    print(frames_dict)
    frame = frames_dict[110]
    cv.imshow("Frame", frame)
    visualize_image("Frame", frame)

    canny_image = canny_edge(frame)
    visualize_image("Canny", canny_image)

    harris_image = canny_edge(frame)
    visualize_image("Harris", harris_image)

{0: array([[[236, 205, 154],
        [236, 205, 154],
        [236, 205, 154],
        ...,
        [154, 180, 204],
        [154, 180, 204],
        [152, 178, 202]],

       [[236, 205, 154],
        [236, 205, 154],
        [236, 205, 154],
        ...,
        [154, 180, 204],
        [154, 180, 204],
        [152, 178, 202]],

       [[236, 205, 154],
        [236, 205, 154],
        [236, 205, 154],
        ...,
        [154, 180, 204],
        [152, 178, 202],
        [152, 178, 202]],

       ...,

       [[ 14,  15,  20],
        [  8,   9,  14],
        [  7,   8,  13],
        ...,
        [191, 198, 211],
        [191, 198, 211],
        [191, 198, 211]],

       [[ 10,  11,  16],
        [  8,   9,  14],
        [  7,   8,  13],
        ...,
        [191, 198, 211],
        [191, 198, 211],
        [191, 198, 211]],

       [[  7,   8,  13],
        [  7,   8,  13],
        [ 10,  11,  16],
        ...,
        [191, 198, 211],
        [191, 198, 211],
        [191, 198, 2

C:\Users\saket\AppData\Local\Temp\ipykernel_5260\2307745397.py:17: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  Image_x = ndimage.filters.convolve(image, Kx)
C:\Users\saket\AppData\Local\Temp\ipykernel_5260\2307745397.py:18: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  Image_y = ndimage.filters.convolve(image, Ky)
